# Greenblatt Magic formula

## The Magic Formula: Explanation and Formula

The **Magic Formula** is an investment strategy developed by **Joel Greenblatt** to identify high-quality companies that are also undervalued. It ranks companies based on two key financial ratios:

---

## 📌 Core Idea

> **Buy good companies at cheap prices.**

The strategy identifies:
- **"Good companies"** → those with high returns on capital (efficient use of capital).
- **"Cheap companies"** → those with high earnings yield (undervalued based on operating profits).

---

## 🧮 The Formula

1. **Earnings Yield (EY):**
$$
\text{Earnings Yield} = \frac{\text{EBIT}}{\text{Enterprise Value}}
$$
- Measures how cheap the stock is.
- EBIT = Earnings Before Interest and Taxes
- Enterprise Value = Market Cap + Debt - Cash

2. **Return on Capital (ROC):**
$$
\text{Return on Capital} = \frac{\text{EBIT}}{\text{Net Working Capital} + \text{Net Fixed Assets}}
$$
- Measures the quality of the business (how efficiently it uses its capital).

---

## 🔍 Implementation Steps

1. **Filter the universe**: Remove financials, utilities, and companies with very small market cap.
2. **Rank all remaining stocks** by:
   - Earnings Yield (high = better)
   - Return on Capital (high = better)
3. **Compute combined rank**:
   $$
   \text{Combined Rank} = \text{Rank}_{EY} + \text{Rank}_{ROC}
   $$
4. **Sort by Combined Rank** (lowest = best overall).
5. **Pick top N stocks** (e.g., top 20–30).
6. **Hold for 1 year**, rebalance annually.

---

## ✅ Why It Works

- Avoids paying too much for popular stocks.
- Focuses on operationally efficient, consistently profitable companies.
- Enforces a disciplined, rules-based approach.

---

## ⚠️ Notes and Caveats

- Avoids subjective judgement; however, **screening accuracy** depends on **quality of financial data**.
- May underperform in short-term or irrational markets.
- Works best over a multi-year horizon (3–5+ years).
- I have added a minimum 40% margin restriction to the original formula. 

---


In [1]:
import os
from tenacity import retry, stop_after_attempt, wait_exponential
import requests
import pandas as pd
from tqdm import tqdm

api_key = os.getenv('financial_modeling_prep_api_key')
assert api_key is not None

In [41]:


# Retry settings: 5 attempts with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=10, max=60))
def fetch_json(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def get_magic_formula(api_key, symbols, minimum_margin=0.4, minimum_marketcap=500_000_000, minimum_revenue=100_000_000):
    import pandas as pd
    from tqdm import tqdm

    base_url = "https://financialmodelingprep.com/stable"
    results = []

    for symbol in tqdm(symbols, desc="Processing symbols"):
        try:
            # Step 1: Check market cap
            ev_url = f"{base_url}/enterprise-values/?symbol={symbol}&apikey={api_key}"
            ev_data = fetch_json(ev_url)[0]
            market_cap = ev_data.get('marketCapitalization')  
            
            if not ev_data or market_cap < minimum_marketcap:
                continue

            # Step 2: Exclude financials and real estate sectors
            profile_url = f"{base_url}/profile?symbol={symbol}&apikey={api_key}"
            profile_data = fetch_json(profile_url)
            sector = profile_data[0].get('sector') 
            company_name = profile_data[0].get('companyName')
            
            if not profile_data or sector in ['Financial Services', 'Real Estate']:
                continue

            # Step 3: Fetch income statement
            income_url = f"{base_url}/income-statement/?symbol={symbol}&apikey={api_key}"
            income_data = fetch_json(income_url)[0]
            net_income = income_data.get('netIncome')
            revenue = income_data.get('revenue')

            # Filter: minimum revenue
            if revenue < minimum_revenue:
                continue

            # Skip if net income is non-positive
            if net_income is None or net_income <= 0:
                continue

            cash_url = f"{base_url}/cash-flow-statement/?symbol={symbol}&apikey={api_key}"
            cash_data = fetch_json(cash_url)[0]
            cashflow = cash_data['operatingCashFlow']
            
            if abs(net_income - cashflow) > 0.5 * net_income:
                print(f"Cashflow mismatch for {symbol}")
                continue  # net income not supported by cash flow

            # Step 4: Fetch balance sheet
            balance_url = f"{base_url}/balance-sheet-statement/?symbol={symbol}&apikey={api_key}"
            balance_data = fetch_json(balance_url)[0]

            # Extract relevant fields
            ebit = income_data.get('ebit')
            
            total_assets = balance_data.get('totalAssets')
            current_liabilities = balance_data.get('totalCurrentLiabilities')
            enterprise_value = ev_data.get('enterpriseValue')
            operating_expenses = income_data.get('operatingExpenses')
            other_expenses = income_data.get('otherExpenses')

            # Ensure all required fields are present
            if None in (ebit, revenue, total_assets, current_liabilities, enterprise_value):
                print(f"missing data for {symbol}")
                continue

            # Filter: operating expenses must be non-negative
            if operating_expenses is not None and operating_expenses < 0:
                continue

            # Filter: exclude extreme accounting adjustments
            if other_expenses is not None and other_expenses < -revenue:
                continue

            # Filter: remove absurdly high EBIT margins (e.g., > 200%)
            ebit_margin = ebit / revenue
            if ebit_margin < minimum_margin or ebit_margin > 2:
                continue

            # Compute Greenblatt metrics
            earnings_yield = ebit / enterprise_value
            capital = total_assets - current_liabilities
            if capital <= 0:
                continue
            return_on_capital = ebit / capital

            # Append result
            results.append({
                'symbol': symbol,
                'company_name': company_name,
                'sector' : sector,
                'ebit_margin': ebit_margin,
                'earnings_yield': earnings_yield,
                'return_on_capital': return_on_capital
            })

        except Exception as e:
            print(f'Symbol: {symbol}\n')
            print(e)

    # Step 6: Compile DataFrame and rank
    df = pd.DataFrame(results)
    if df.empty:
        return df

    df['ey_rank'] = df['earnings_yield'].rank(ascending=False)
    df['roc_rank'] = df['return_on_capital'].rank(ascending=False)
    df['combined_rank'] = df['ey_rank'] + df['roc_rank']
    df_sorted = df.sort_values(by='combined_rank')

    return df_sorted #[['symbol', 'ebit_margin', 'earnings_yield', 'return_on_capital', 'combined_rank']]


In [46]:

symbols = pd.read_csv("russel1000.csv")
symbols = symbols['Ticker'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.1)
print(top_stocks)
russel1000_filtered = top_stocks

Processing symbols: 100%|█████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  2.02it/s]

  symbol          company_name                  sector  ebit_margin  \
1   GOOG         Alphabet Inc.  Communication Services     0.343077   
2   NVDA    NVIDIA Corporation              Technology     0.645785   
0   META  Meta Platforms, Inc.  Communication Services     0.433906   

   earnings_yield  return_on_capital  ey_rank  roc_rank  combined_rank  
1        0.051450           0.332516      1.0       2.0            3.0  
2        0.028965           0.900795      3.0       1.0            4.0  
0        0.047942           0.294393      2.0       3.0            5.0  


In [47]:
russel1000_filtered.head(50)

,symbol,company_name,sector,ebit_margin,earnings_yield,return_on_capital,ey_rank,roc_rank,combined_rank
1,GOOG,Alphabet Inc.,Communication Services,0.343077,0.051450,0.332516,1.0,2.0,3.0
2,NVDA,NVIDIA Corporation,Technology,0.645785,0.028965,0.900795,3.0,1.0,4.0
0,META,"Meta Platforms, Inc.",Communication Services,0.433906,0.047942,0.294393,2.0,3.0,5.0


In [ ]:
symbols = pd.read_csv("all_listed_companies.csv")
symbols = symbols['Symbol'].to_list()

top_stocks = get_magic_formula(api_key, symbols, minimum_margin=0.1)
print(top_stocks)
world_filtered = top_stocks

In [48]:
world_filtered.head(40)

,symbol,ebit_margin,earnings_yield,return_on_capital,combined_rank
74,KSPI,0.504203,0.140632,0.782014,42.0
225,EQNR,0.312609,0.360830,0.336965,46.0
293,MO,0.723048,0.132416,0.560009,52.0
34,CVAC,0.356667,0.644654,0.261309,67.0
254,HAFN,0.270932,0.206253,0.254951,85.0
153,ANF,0.157773,0.124960,0.359307,85.0
333,PHM,0.223233,0.170498,0.252484,96.0
387,TNK,0.334769,0.447610,0.223431,99.0
179,CIG,0.260321,0.246088,0.227419,101.0
157,ASR,0.676595,0.137092,0.272665,104.0


## Tests

In [39]:
base_url = "https://financialmodelingprep.com/stable"
profile_url = f"{base_url}/search-exchange-variants?symbol=HES&apikey={api_key}"
response = requests.get(profile_url)
response.raise_for_status()
profile_data = response.json()

In [40]:
profile_data

[{'symbol': 'HES',
  'price': 132.19,
  'beta': 0.623,
  'volAvg': 2003465,
  'mktCap': 40882533490,
  'lastDiv': 2,
  'range': '123.79-161.69',
  'changes': -1.43,
  'companyName': 'Hess Corporation',
  'currency': 'USD',
  'cik': '0000004447',
  'isin': 'US42809H1077',
  'cusip': '42809H107',
  'exchange': 'New York Stock Exchange',
  'exchangeShortName': 'NYSE',
  'industry': 'Oil & Gas Exploration & Production',
  'website': 'https://www.hess.com',
  'description': 'Hess Corporation, an exploration and production company, explores, develops, produces, purchases, transports, and sells crude oil, natural gas liquids (NGLs), and natural gas. The company operates in two segments, Exploration and Production, and Midstream. It conducts production operations primarily in the United States, Guyana, the Malaysia/Thailand Joint Development Area, and Malaysia; and exploration activities principally offshore Guyana, the U.S. Gulf of Mexico, and offshore Suriname and Canada. The company is also

In [ ]:
top_stocks